In [1]:
import sqlite3

import pandas as pd

In [2]:
df_ad = pd.read_csv('data/ad.csv')
df_event = pd.read_csv('data/event.csv')
df_event_location = pd.read_csv('data/event_location.csv')

In [3]:
conn = sqlite3.connect(':memory:')

In [4]:
df_ad.to_sql('ad', conn, index=False)
df_event.to_sql('event', conn, index=False)
df_event_location.to_sql('event_location', conn, index=False)

3515

### What is the percentage of ads from a Hungarian event location?

In [5]:
query1 = """
SELECT ROUND((SUM(CASE WHEN event_location.country = 'HU' THEN 1 ELSE 0 END) * 100.0) /
             COUNT(*), 2) AS percentage_hu_ads
FROM ad
         JOIN event ON ad.event_id = event.id
         JOIN event_location ON event.location_id = event_location.id;
"""

In [6]:
result1 = pd.read_sql_query(query1, conn)
result1

,percentage_hu_ads
0,3.05


### What is the average original price of events in the Netherlands?

2a is the query for when we want to take the average price of all ads in the Netherlands.
2b is the query for when we want to first calculate the average price per event and then take the average of those averages.

I was not sure which one was asked, so I included both. My guess would be that 2b is asked as it makes more sense.
I elaborate on why in the `answers/question_2/2.sql` file.

In [7]:
query2a= """
SELECT ROUND(AVG(ad.original_price), 2) AS avg_org_price_nl
FROM ad
         JOIN event ON ad.event_id = event.id
         JOIN event_location ON event.location_id = event_location.id
WHERE event_location.country = 'NL';
"""

In [8]:
result2a = pd.read_sql_query(query2a, conn)
result2a

,avg_org_price_nl
0,4666.44


In [9]:
query2b = """
SELECT ROUND(AVG(event_avg_org_price), 2) AS avg_org_price_nl
FROM (SELECT event.id               AS event_id,
             AVG(ad.original_price) AS event_avg_org_price
      FROM ad
               JOIN event ON ad.event_id = event.id
               JOIN event_location ON event.location_id = event_location.id
      WHERE event_location.country = 'NL'
      GROUP BY event.id) AS avg_org_price_per_event_nl;
"""

In [10]:
result2b = pd.read_sql_query(query2b, conn)
result2b

,avg_org_price_nl
0,3256.15


### What was the number of sold listings for Pixies concerts?

In [11]:
query3 = """
SELECT COUNT(*) AS sold_pixies_listings
FROM ad
         JOIN event ON ad.event_id = event.id
WHERE event.title LIKE '%Pixies%'
  AND ad.is_sold = 1;
"""

In [12]:
result3 = pd.read_sql_query(query3, conn)
result3

,sold_pixies_listings
0,25
